In [3]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [118]:
PROBLEM_SIZE = 10
NUM_SETS = 15
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [119]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

def distance_old(state):
    return PROBLEM_SIZE - np.sum(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [120]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [23]:
def f(state):
    return len(state.taken)

In [52]:
def number_of_special_sets(state):
    sets = [SETS[i] for i in range(len(state.not_taken))]
    counts = np.sum(sets, axis=0)
    count = list(counts).count(1)
    return count

In [68]:
def find_set_covers(state):
    coverings = {i: [] for i in range(PROBLEM_SIZE)}
    #sets_left = list(state.not_taken)
    sets = [SETS[element] for element in state.not_taken]
    for i in range(len(sets)):
        for j in range(PROBLEM_SIZE):
            if (sets[i][j]):
                coverings[j].append(i)
    return coverings


In [69]:
def get_special_sets(state):
    special_sets = []
    coverings = find_set_covers(state)  
    for value in coverings.values():
        if(len(value) == 1):
            special_sets.append(value[0])
    return special_sets

In [108]:
def h(state):
    special_sets = get_special_sets(state)
    for special_set in special_sets:
        if (goal_check(State(state.taken ^ {special_sets[0]}, state.not_taken ^ {special_sets[0]}))):
            return 1
    if (len(special_sets) > 0):
        return len(special_sets) + 1
    return distance_old(state)

In [97]:
def distance(state):
    return f(state) + h(state)

In [98]:
"""
Defines a class: priority_queue that implements the methods put and get.
Methods: Put() and Get()
"""

class protiorty_queue():
    queue = []
    
    # The Put method simply adds a state to the queue
    def put(self, element):
        self.queue.append(element)
    
    # The get method returns the set with the highest number of covered points not in the current state
    def get(self):
        shortest = np.inf
        best_action = State(set(), set(range(NUM_SETS)))
        for element in self.queue:
            sets = [SETS[i] for i in element.taken]
            if (not len(sets)):
                break
            covered = np.sum(reduce(np.logical_or, sets))
            if (PROBLEM_SIZE -  covered < shortest):
                shortest = PROBLEM_SIZE - covered
                best_action = element
        self.queue.remove(best_action)
        return best_action


In [121]:
# frontier = PriorityQueue()
frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance_old(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((distance_old(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 1,446 steps (3 tiles)


In [117]:
get_special_sets(state)

[]